<a href="https://colab.research.google.com/github/t-willi/Simula/blob/main/PTB_normal_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb
!pip install ecg_plot
!pip install wfdb

     |████████████████████████████████| 157 kB 27.4 MB/s 
     |████████████████████████████████| 157 kB 25.6 MB/s 
     |████████████████████████████████| 157 kB 68.6 MB/s 
     |████████████████████████████████| 157 kB 53.1 MB/s 
     |████████████████████████████████| 157 kB 75.8 MB/s 
     |████████████████████████████████| 157 kB 58.7 MB/s 
     |████████████████████████████████| 156 kB 47.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=cc61ac6d6d47e396464f4d8389b4a7359874407fc70ee0f25f243cfa8f1c1dd4
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 161 kB 18.5 MB/s 


In [ ]:
import pathlib
from pathlib import Path
# current_position=pathlib.PurePath(__file__)
# dir=current_position.parent
# main_folder=dir.joinpath("main_folder")
current_position=pathlib.PurePath("/content/")
main_folder=current_position.joinpath("main_folder")
Path(main_folder).mkdir(parents=False,exist_ok = True)
#create folder and directory for artifacts
artifact_dir=main_folder.joinpath("artifacts")
Path(artifact_dir).mkdir(parents=False,exist_ok = True)
#create folder and directory for train_data
train_dir=main_folder.joinpath("train_dir")
Path(train_dir).mkdir(parents=False,exist_ok = True)
#create folder and directory for ecg_files
ecg_dir=main_folder.joinpath("ecg")
Path(ecg_dir).mkdir(parents=False,exist_ok = True)
#create folder and directory for saved model sate dicts
model_dir=main_folder.joinpath("model")
Path(model_dir).mkdir(parents=False,exist_ok = True)


In [ ]:
import wandb
import pandas as pd
import torch
from torch import nn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
import torch.optim as optim
from random import shuffle
from tqdm.auto import tqdm
import requests
import zipfile
from pathlib import Path

if torch.cuda.is_available()==True:
  device="cuda:0"
else:
  device ="cpu"
wandb.login(key="7a8ee9d41cc2d51eb77fd795e14f74a215e63c2d")
api = wandb.Api()
artifact = api.artifact('ecg_simula/upload_PTB_XL_normal.zip/PTB_normal:v0', type='dataset')
artifact.download(artifact_dir)
model = api.artifact('ecg_simula/AE_pTOP_serverrun_synthetic_data/Model:v10', type='Model')
model.download(artifact_dir)
torch.manual_seed(42)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Downloading large artifact PTB_normal:v0, 802.52MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:9.3
wandb:   1 of 1 files downloaded.  


In [ ]:
# previous_model = api.artifact('ecg_simula/AE_pTOP_LS_optim/Model:v8', type='Model')
# previous_model.download()

def request(path=None,name=None):
  import requests
  from pathlib import Path
  request = requests.get(path)
  name=name+".py"
  with open(name,"wb") as f:
    f.write(request.content)

unzip_git_dir="https://raw.githubusercontent.com/t-willi/NeuralNetworks/main/unzip.py"
Dataloader_git_dir="https://raw.githubusercontent.com/t-willi/NeuralNetworks/main/dataset_and_loader.py"
get_pred_no_reshape_git_dir="https://raw.githubusercontent.com/t-willi/NeuralNetworks/main/get_predictions_no_reshape.py"
get_pred_git_dir="https://raw.githubusercontent.com/t-willi/NeuralNetworks/main/get_predictions.py"
plt_ECG_git_dir="https://raw.githubusercontent.com/t-willi/NeuralNetworks/main/plot_ECG.py"

request(unzip_git_dir,"Unzip")
from Unzip import unzip

request(Dataloader_git_dir,"dataset_and_loader")
from dataset_and_loader import Custom_dataset as CD
from dataset_and_loader import make_loader as ml



#download prediction generator
request(get_pred_no_reshape_git_dir,"get_predictions")
from get_predictions import get_pred
#download ECG plotter
request(plt_ECG_git_dir,"plot_ECG")
from plot_ECG import plotECG

artifact_dir_str=str(artifact_dir.joinpath("PTB_normal.zip"))
unzip(save_path=train_dir,zip_path=artifact_dir_str,reload=True)

/content/main_folder/train_dir directory exists.
Unzipping data to folder...
unzip is finished


In [ ]:
import glob
import pandas as pd
import torch
# change mx_value setting to max value of data to normalaize between 1 and -1
class Custom_dataset():
    def __init__(self, data_dir,max_value=33,column=3,split=True,target="train",size=1):
      #get all files from directory loaded in all_files list
      self.column=column
      self.max_value=max_value
      self.size=size
      #should shuffle the data here?
      self.files = glob.glob(data_dir + '/*.csv')
      self.len=int((len(self.files))*self.size)
      #print(f"len:{self.len}")
      self.cut1=int(self.len*0.8)
      #print(f"cut1:{self.cut1}")
      self.cut2=int(self.len*0.9)
      #print(f"cut2:{self.cut2}")
      self.train_files=self.files[0:self.cut1]
      self.test_files=self.files[self.cut1:self.cut2]
      self.val_files=self.files[self.cut2:self.len]
      self.target=target
      self.split=split

    def __len__(self):
      if self.split is True:
        if self.target is "train":
          return len(self.train_files)
        if self.target is "test":
          return len(self.test_files)
        if self.target is "val":
          return len(self.val_files)
      if self.split is not True:
        return len(self.files)

    def __getitem__(self,idx):
      header=["I", "II", "III", "aVF", "aVR", "aVL", "V1", "V2", "V3", "V4", "V5", "V6"]
      #turn list of dataframes into Tensor
      if self.split is True:
        if self.target is "train":
          temp_df=pd.read_csv(self.train_files[idx],index_col=0,header=0,names=header)
        if self.target is "test":
          temp_df=pd.read_csv(self.test_files[idx],index_col=0,header=0,names=header)
        if self.target is "val":
          temp_df=pd.read_csv(self.val_files[idx],index_col=0,header=0,names=header)
      if self.split is not True:
        temp_df=pd.read_csv(self.files[idx],index_col=0,header=0,names=header)
      temp_df/=self.max_value
      #load input tensor
      
      temp_list_in=temp_df.iloc[:,0]
      #temp_list_in=normalize([temp_list_in], norm="max")
      temp_tensor_in = torch.tensor(temp_list_in,dtype=torch.float32)
      temp_tensor_in=temp_tensor_in.unsqueeze(0)
      #load label Tensor
      temp_list_out=temp_df.iloc[:,[1,6,7,8,9,10,11]].values
      #temp_list_out=normalize([temp_list_out], norm="max")
      temp_tensor_out=torch.tensor(temp_list_out,dtype=torch.float32)
      temp_tensor_out=temp_tensor_out.T
      #combine input and label and output
      temp_tensor_pair= temp_tensor_in,temp_tensor_out
      return temp_tensor_pair

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data

class Transpose1dLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding=11, upsample=None, output_padding=1):
        super(Transpose1dLayer, self).__init__()
        self.upsample = upsample
        self.upsample_layer = torch.nn.Upsample(scale_factor=upsample)
        reflection_pad = kernel_size // 2
        self.reflection_pad = nn.ConstantPad1d(reflection_pad, value=0)
        self.conv1d = torch.nn.Conv1d(in_channels, out_channels, kernel_size, stride)
        self.Conv1dTrans = nn.ConvTranspose1d(in_channels, out_channels, kernel_size, stride, padding, output_padding)

    def forward(self, x):
        if self.upsample:
            #x = torch.cat((x, in_feature), 1)
            return self.conv1d(self.reflection_pad(self.upsample_layer(x)))
        else:
            return self.Conv1dTrans(x)


class Pulse2pulseGenerator(nn.Module):
    def __init__(self,latent_dim=100, post_proc_filt_len=512,upsample=True):
        super(Pulse2pulseGenerator, self).__init__()
        # "Dense" is the same meaning as fully connection.
        stride = 4
        if upsample:
            stride = 1
            upsample = 5
        # if upsample is anything but none Transpose1dLayer will do
        # self.conv1d(self.reflection_pad(self.upsample_layer(x)))
        # which is a 1d convolution on padded and upsampled data x
        self.deconv_1 = Transpose1dLayer(250 , 250, 25, stride, upsample=upsample)
        self.deconv_2 = Transpose1dLayer(250, 150, 25, stride, upsample=upsample)
        self.deconv_3 = Transpose1dLayer(150, 50, 25, stride, upsample=upsample)
        self.deconv_4 = Transpose1dLayer( 50, 25, 25, stride, upsample=2)
        self.deconv_5 = Transpose1dLayer( 25, 10, 25, stride, upsample=upsample)
        self.deconv_6 = Transpose1dLayer(  10, 7, 25, stride, upsample=2)


        #new convolutional layers
        self.conv_1 = nn.Conv1d(1, 10, 25, stride=2, padding=25 // 2)
        self.conv_2 = nn.Conv1d(10, 25, 25, stride=5, padding= 25 // 2)
        self.conv_3 = nn.Conv1d(25, 50 , 25, stride=2, padding= 25 // 2)
        self.conv_4 = nn.Conv1d(50, 150 , 25, stride=5, padding= 25 // 2)
        self.conv_5 = nn.Conv1d(150, 250 , 25, stride=5, padding= 25 // 2)
        self.conv_6 = nn.Conv1d(250, 250 , 25, stride=5, padding= 25 // 2)
        self.flatt = nn.Flatten()
        self.linear1 = nn.Linear(500,100)
        self.linear2 = nn.Linear(100,500)

        for m in self.modules():
            if isinstance(m, nn.ConvTranspose1d) or isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight.data)

    def forward(self, x, LS=False):
        self.LS=LS
        if x.ndim==2:
          x=x.unsqueeze(0)
        x = F.leaky_relu(self.conv_1(x)) #(1,1,5000 --> 1, 10, 2500)
        x = F.leaky_relu(self.conv_2(x)) #( --> 1, 25, 500)
        x = F.leaky_relu(self.conv_3(x)) #(--> 1, 50, 250)
        x = F.leaky_relu(self.conv_4(x)) # --> 1, 150, 50)
        x = F.leaky_relu(self.conv_5(x)) #(--> 1, 250, 10)
        x = F.leaky_relu(self.conv_6(x)) #(--> 1, 250, 2)-->flatten into (1,500)), then to linear ((1,100)), and then back
        x = self.flatt(x) # (1,500)
        LS = self.linear1(x) #(1,100)
        if self.LS is True:
          return LS
        x = self.linear2(LS) #(1,500)
        zero_dim=x.shape[0]
        x=torch.reshape(x,(zero_dim,250,2)) #1(1,250,2)
        x = F.relu(self.deconv_1(x)) #(--> 1, 250, 10)
        x = F.relu(self.deconv_2(x)) #(--> 1, 150, 50)
        x = F.relu(self.deconv_3(x)) #( --> 1, 50, 250)
        x = F.relu(self.deconv_4(x)) #(--> 1, 25, 500)
        x = F.relu(self.deconv_5(x)) #(--> 1, 10, 2500)
        x = torch.tanh(self.deconv_6(x)) #(1, 7, 5000)
        x=x.squeeze()
        return x

model=Pulse2pulseGenerator().to(device)
artifact_dir_string=str(artifact_dir.joinpath("model"))
model_path= artifact_dir_string
model.load_state_dict(torch.load(model_path, map_location=device))

<All keys matched successfully>

In [ ]:
import wandb

# 🐝 initialise a wandb run
config = dict(
    epochs=20,
    batch_size=32,
    learning_rate=0.0001,)  #learing rate from puls to puls paper


def model_pipeline(hyperparameters,model=model):
    # tell wandb to get started
    wandb.init(project="AE_pTOP_PTB_Normal_colab_test", config=hyperparameters)
    # access all HPs through wandb.config, so logging matches execution!
    config = wandb.config
    # make the model, data, and optimization problem
    train_loader, val_loader,test_dataset, criterion, optimizer,val_dataset = make(config)
    # and use them to train the model
    train(model, train_loader,val_loader,test_dataset, criterion, optimizer,val_dataset,config)
    return model

In [ ]:
def make(config):
    # Make the data
    print("making data")
    data_dir=str(train_dir)
    train_dataset = Custom_dataset(data_dir=data_dir,split=True,target="train",size=1)
    val_dataset = Custom_dataset(data_dir=data_dir,split=True,target="val",size=1)
    test_dataset = Custom_dataset(data_dir=data_dir,split=True,target="test",size=1)
    train_loader = ml(train_dataset, batch_size=config.batch_size)
    val_loader = ml(val_dataset, batch_size=config.batch_size)
    

    # Make the loss and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
    
    return train_loader, val_loader,test_dataset, criterion, optimizer,val_dataset


In [ ]:
# import torch
# import pandas as pd

# def get_pred(dataset=None,Set=None,model=None,random=True,upscale=None):
#   """
#   Function takes a Tensor Dataset as input,first a random file from the dataset is selected,
#   then the Tensor pair is recombined and shaped into a df-->df_Input. 
#   X is used afterwards as input into the model. The predictions are safed as --> df_output.
#   Both dataframes are now unscaled by 5011, the max value of the whole dataset.
#   Then a tuple pair of input and output is returned.
#   """
#   if random:
#     import random
#     limit=len(dataset)
#     rand_idx=random.randint(0,limit)
#     X,y=dataset[rand_idx]
#   if random is False:
#       X,y=Set
#   #need to combine tensors to make dataframe for plotting input and output side by side
#   full_tensor=torch.cat((X,y.squeeze()))
#   full_tensor=full_tensor.numpy()
#   df_input=pd.DataFrame(full_tensor).T
#   df_input.columns = ["R1","R2","Rv1","Rv2","Rv3","Rv4","Rv5","Rv6"]
#   model.to("cpu")
#   model.eval()
#   with torch.inference_mode():
#     output=model(X)
#   output=output.detach().numpy()
#   output=output.squeeze().T
#   #unscale data
#   df_output = pd.DataFrame(output,columns=["F2","Fv1","Fv2","Fv3","Fv4","Fv5","Fv6"])
#   if upscale:
#     df_input=df_input*upscale
#     df_output=df_output*upscale
#   return df_input,df_output

In [ ]:
def train(model, train_loader,val_loader,test_dataset, criterion, optimizer,val_dataset, config):
  # Tell wandb to watch what the model gets up to: gradients, weights, and more!
  wandb.watch(model, criterion, log="all")
  for epoch in tqdm((range(config.epochs))):
    train_loss=0
    for batch,(X,y) in tqdm((enumerate(train_loader))):
      # Forward pass ➡
      X, y = X.to(device), y.to(device)
      #print(f"shape of input{x.shape},shape of label_y{y.shape}") 
      model.train()
      #print(f"x.shape is{X.max()} y.shape is {y.max()}")
      output=model(X)
      #print(f"output is {output.max()}")
      #print(f"shape of model_output_raw{output.shape}") 
      # output=torch.reshape(output,(config.batch_size, 1, 7, 5000))
      loss = criterion(output,y)
      train_loss += loss
      # Backward pass ⬅
      optimizer.zero_grad()
      loss.backward()
      # Step with optimizer
      optimizer.step()
    #average loss per batch
    train_loss /= len(train_loader)


    val_loss = 0
    model.eval()
    with torch.inference_mode():
      for batch,(X,y) in tqdm(enumerate(val_loader)):
        #print("doing test loop")
        X, y = X.to(device), y.to(device)
        val_pred = model(X)
        # val_pred=torch.reshape(val_pred,(config.batch_size, 1, 7, 5000))
        loss=criterion(val_pred,y)
        val_loss += loss
      val_loss /= len(val_loader)  
      wandb.log({"train_loss": train_loss, 
                 "val_loss": val_loss,
                 "Epoch":epoch})
      

    if (epoch) % 1==0:
      df_input,df_output=get_pred(dataset=val_dataset,model=model,random=True,upscale=33)
      model.to(device)
      #plotting the ECG and creating the combined DF
      combined_df=plotECG(df_input,df_output,path=str(ecg_dir))
      #saving combined DF as table on wandB
      input_prediction_table = wandb.Table(dataframe=combined_df)
      ecg_dir_file=ecg_dir.joinpath("ecg.png")
      wandb.log({"ECG": wandb.Image(str(ecg_dir_file))})
      wandb.log({"Input and predictions": input_prediction_table}) 
      
    if (epoch) % 1==0:
      print("one")
      model_dir_model=model_dir.joinpath("model")
      torch.save(model.state_dict(),str(model_dir_model))
      print("two")
      wandb.log_artifact(str(model_dir_model), name='Model', type='Model')

In [ ]:
model = model_pipeline(config)

Epoch,▁
train_loss,▁
val_loss,▁
Epoch,0
train_loss,1e-05
val_loss,0.0


making data


  0%|          | 0/20 [00:00<?, ?it/s]

0it [00:00, ?it/s]